In [2]:
#Setting up the environment

!java -version

#Install Spark
#download file
!wget -q http://apache.osuosl.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
#extract the file
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
#install findspark package
!pip install -q findspark

openjdk version "11.0.15" 2022-04-19
OpenJDK Runtime Environment (build 11.0.15+10-Ubuntu-0ubuntu0.20.04.1)
OpenJDK 64-Bit Server VM (build 11.0.15+10-Ubuntu-0ubuntu0.20.04.1, mixed mode, sharing)


In [3]:
# 

import os
pathToSpark = "/mnt/c/Users/walid/Desktop/Big Data/Final/spark-3.2.1-bin-hadoop3.2"
os.environ["SPARK_HOME"] = pathToSpark

In [2]:
import findspark
findspark.init()

# create entry points to spark
try:
    sc.stop()
except:
    pass
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = SparkConf().setAppName("FinalProject").setMaster("local[*]")
sc=SparkContext(conf = conf)
spark = SparkSession(sparkContext=sc)

In [3]:
# Reading in the directory containing all the JSON files and storing them into
# Spark's RDD format

import time

Filepath = "/mnt/c/Users/walid/Desktop/Big Data/Final/mpdata/*.json"

start = time.time()

df = spark.read.option("multiline", "true").json(Filepath)

end = time.time()
total_time = end-start

print("took " + str(total_time) + " seconds")

took 42.50797080993652 seconds


In [4]:
# Taking a look at the schema, we see that all of the headers were successfully imported and 
# we have all of the data we need to begin analyzing
#
# The format of the JSON files is deeply nested and so we only have two main columns:
# (1) info regarding the slice/part of the json files and when it was generated... info that is useless to us
# and (2) a column of playlists, which consists of rows of arrays, meaning each row contains multiple playlists
# 
# We will have to flatten and normalize this dataframe first before we can analyze the data

df.show(5)
df.printSchema()

+--------------------+--------------------+
|                info|           playlists|
+--------------------+--------------------+
|{2017-12-04 03:05...|[{false, null, 82...|
|{2017-12-03 08:41...|[{false, null, 10...|
|{2017-12-04 03:05...|[{false, null, 16...|
|{2017-12-04 03:05...|[{false, null, 39...|
|{2017-12-04 03:05...|[{false, null, 82...|
+--------------------+--------------------+
only showing top 5 rows

root
 |-- info: struct (nullable = true)
 |    |-- generated_on: string (nullable = true)
 |    |-- slice: string (nullable = true)
 |    |-- version: string (nullable = true)
 |-- playlists: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- collaborative: string (nullable = true)
 |    |    |-- description: string (nullable = true)
 |    |    |-- duration_ms: long (nullable = true)
 |    |    |-- modified_at: long (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- num_albums: long (nullable = true)
 |    |  

In [28]:
from pyspark.sql import functions as F
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import StopWordsRemover

In [6]:
# Here, we've removed the useless info column and exploded the array of playlists so that
# each playlist was it's own row. 
#
# To double check, we counted the number of rows (1,000,000 playlists)

df = df.select(F.explode(F.col("playlists")).alias("playlists"))
#df1.printSchema()
df.show()
print("counting # of playlists...")
df.count()

+--------------------+
|           playlists|
+--------------------+
|{false, null, 824...|
|{false, null, 278...|
|{false, null, 275...|
|{false, null, 178...|
|{false, null, 243...|
|{false, null, 134...|
|{false, null, 523...|
|{false, null, 238...|
|{false, null, 805...|
|{false, null, 279...|
|{false, null, 872...|
|{false, null, 977...|
|{false, really gr...|
|{false, null, 293...|
|{false, null, 200...|
|{false, null, 797...|
|{false, null, 865...|
|{false, null, 939...|
|{false, null, 667...|
|{false, null, 434...|
+--------------------+
only showing top 20 rows

counting # of playlists...


1000000

In [ ]:
# Here we take the "playlists" column exploded from earlier and select every column to be mapped to its proper header
# 
# We can now work with the playlists data to analyze, calculate, and answer questions.

df = df.select("playlists.*") #TODO: change * to only columns that we need to speed up operation times
df.show(3)
df.printSchema()

In [8]:
# Here we isolate the tracks column, which has a lot of information about the tracks nested inside arrays
# we explode it and only select the columns that we are interested in looking at...

df_tracks = df.select("tracks")
df_tracks = df_tracks.select(F.explode(F.col("tracks")).alias("tracks"))
df_tracks = df_tracks.select("tracks.album_name", "tracks.artist_name", "tracks.track_name", "tracks.duration_ms")
df_tracks.show(10)
df_tracks.count()

+--------------------+--------------+--------------------+-----------+
|          album_name|   artist_name|          track_name|duration_ms|
+--------------------+--------------+--------------------+-----------+
|What's up (feat. ...|   Post Malone|What's up (feat. ...|     290533|
|     T R A P S O U L| Bryson Tiller|               Don't|     198293|
|    Remember My Name|      Lil Durk|             Like Me|     238439|
|            Barter 6|    Young Thug|               Check|     230693|
|            Barter 6|    Young Thug|With That (feat. ...|     202533|
|         Best Friend|    Young Thug|         Best Friend|     213000|
|good kid, m.A.A.d...|Kendrick Lamar|  Backseat Freestyle|     212653|
|Dreams Worth More...|     Meek Mill|R.I.C.O. (feat. D...|     197133|
|           SremmLife|  Rae Sremmurd|             No Type|     200080|
|Quarterback (feat...|          Thug|Quarterback (feat...|     301767|
+--------------------+--------------+--------------------+-----------+
only s

66346428

# Track Analysis
---

In [9]:
 # Q1: Who were the most popular artists in 2010-2017 in users' playlists?

df_tracks.groupBy("artist_name").count().orderBy(['count'], ascending=[False]).show()

+-----------------+------+
|      artist_name| count|
+-----------------+------+
|            Drake|847160|
|       Kanye West|413297|
|   Kendrick Lamar|353624|
|          Rihanna|339570|
|       The Weeknd|316603|
|           Eminem|294667|
|       Ed Sheeran|272116|
|           Future|250734|
|    Justin Bieber|243119|
|          J. Cole|241560|
|          Beyoncé|230857|
| The Chainsmokers|223509|
|      Chris Brown|212772|
|    Calvin Harris|203047|
|Twenty One Pilots|198905|
|     Lil Uzi Vert|197855|
|      Post Malone|195907|
|         Big Sean|192478|
|         Maroon 5|187029|
|            JAY Z|185520|
+-----------------+------+
only showing top 20 rows



In [11]:
# Q2: What were the most popular tracks in 2010-2017 in users' playlists?

df_tracks.groupBy("track_name", "artist_name").count().orderBy(['count'], ascending=[False]).show()

+--------------------+-----------------+-----+
|          track_name|      artist_name|count|
+--------------------+-----------------+-----+
|             HUMBLE.|   Kendrick Lamar|46574|
|           One Dance|            Drake|43447|
|Broccoli (feat. L...|             DRAM|41309|
|              Closer| The Chainsmokers|41079|
|     Congratulations|      Post Malone|39987|
|            Caroline|            Aminé|35202|
|iSpy (feat. Lil Y...|             KYLE|35138|
|Bad and Boujee (f...|            Migos|34999|
|            Location|           Khalid|34990|
|       XO TOUR Llif3|     Lil Uzi Vert|34922|
|         Bounce Back|         Big Sean|33699|
|    Ignition - Remix|         R. Kelly|32391|
|      No Role Modelz|          J. Cole|32336|
|            Mask Off|           Future|32059|
|No Problem (feat....|Chance The Rapper|31492|
|         I'm the One|        DJ Khaled|31374|
|             Jumpman|            Drake|31119|
|          goosebumps|     Travis Scott|31106|
|           F

In [52]:
# Q3: What is the average song duration? What is the longest song? Shortest?

# TODO: convert avg song to mm:ss
# TODO: filter -1/0 values from duration_ms
# TODO: investigate why max song is so high

df_tracks.select((F.avg("duration_ms")*60000).alias("Avg Song Duration"), \
                 (F.max("duration_ms")*60000).alias("Max Song Duration"), \
                 (F.min("duration_ms")*60000).alias("Min Song Duration")).show()

+--------------------+-----------------+-----------------+
|   Avg Song Duration|Max Song Duration|Min Song Duration|
+--------------------+-----------------+-----------------+
|1.408938866402694...|    1244674500000|           -60000|
+--------------------+-----------------+-----------------+



# Playlist Analysis
---

In [32]:
df_playlists = df.select("pid", "name", "description", "num_tracks", "num_artists", "num_albums", "duration_ms", "tracks")

In [14]:
# Q1: What is the most common word in the playlist name?

df_playlists.withColumn('name', F.explode(F.split(F.col('name'), ' '))) \
  .groupBy('name') \
  .count() \
  .sort('count', ascending=False) \
  .show()

+---------+-----+
|     name|count|
+---------+-----+
|         |70727|
|  Country|13280|
|    Music|13031|
|     Rock|12369|
|    music|12356|
|   Summer|12298|
|    songs|11945|
|     2017|11543|
|    Chill|10976|
|     2016| 9966|
| Playlist| 9325|
|    Party| 8977|
|Christmas| 8678|
|    chill| 8392|
|  country| 8323|
|     Good| 8162|
|      New| 8136|
|      Rap| 8033|
|   summer| 7832|
|      The| 7806|
+---------+-----+
only showing top 20 rows



In [15]:
# Q2: What is the most common word in the playlist description?

df_playlists.withColumn('description', F.explode(F.split(F.col('description'), ' '))) \
  .groupBy('description') \
  .count() \
  .sort('count', ascending=False) \
  .show()

+-----------+-----+
|description|count|
+-----------+-----+
|        the| 4501|
|         to| 3929|
|           | 3386|
|        and| 3312|
|         of| 3120|
|          a| 2523|
|        for| 2229|
|      songs| 2210|
|        you| 2150|
|       that| 1598|
|         in| 1561|
|         my| 1484|
|          I| 1391|
|   playlist| 1281|
|         is| 1279|
|       this| 1132|
|          i| 1087|
|      music| 1066|
|       your|  955|
|       with|  908|
+-----------+-----+
only showing top 20 rows



In [56]:
# The most common words are not really useful here in the description,
# So some filtering must be done to remove the common word and find common KEY words

# TODO: Add more filters or find a way to add values into array and filter by that

pl_clean_description = df_playlists.withColumn('words', F.explode(F.split(F.col('description'), ' '))) \
  .groupBy('words') \
  .count() \
  .filter((F.col("words") != "the") & (F.col("words") != "to") & (F.col("words") != " ") & (F.col("words") != "and") &\
          (F.col("words") != "of") & (F.col("words") != "a") & (F.col("words") != "for") & (F.col("words") != "you") &\
          (F.col("words") != "that") & (F.col("words") != "in") & (F.col("words") != "my") & (F.col("words") != "I") &\
          (F.col("words") != "is") & (F.col("words") != "this") & (F.col("words") != "i") & (F.col("words") != "your")&\
          (F.col("words") != "with") & (F.col("words") != "from") & (F.col("words") != "all") & (F.col("words") != "me")&\
          (F.col("words") != "some") & (F.col("words") != "on") & (F.col("words") != "when") & (F.col("words") != "it")&\
          (F.col("words") != "just") & (F.col("words") != "like") & (F.col("words") != "are") & (F.col("words") != "or")&\
          (F.col("words") != "The") & (F.col("words") != "be") & (F.col("words") != " ") & (F.col("words") != "A")&\
          (F.col("words") != "have") & (F.col("words") != "at") & (F.col("words") != "these") & (F.col("words") != "I") &\
          (F.col("words") != "that") & (F.col("words") != "in") & (F.col("words") != "my") & (F.col("words") != "I") &\
          (F.col("words") != "but") & (F.col("words") != "get") & (F.col("words") != "by") & (F.col("words") != "not")) \
  .sort('count', ascending=False) \
  
pl_clean_description.show()

+--------+-----+
|   words|count|
+--------+-----+
|        | 3386|
|   songs| 2210|
|playlist| 1281|
|   music| 1066|
|    good|  611|
|    love|  547|
|  listen|  500|
|    best|  469|
|   Songs|  429|
|    make|  418|
|favorite|  376|
|     out|  355|
|    This|  342|
|    feel|  335|
|      so|  332|
|      up|  329|
|   chill|  326|
|    will|  324|
|       -|  322|
|      as|  306|
+--------+-----+
only showing top 20 rows



In [ ]:
#list_of_genres = {"pop", "rock", "hip hop", "latin", "dance", "edm", "r&b", "country", "classical", "metal", "jazz"}

In [53]:
# Q3: What is the average playlist duration? Is there a maximum duration to a playlist? min?

df_playlists.select((F.avg("duration_ms")*60000).alias("Avg Playlist Duration"), \
                    (F.max("duration_ms")*60000).alias("Max Playlist Duration"), \
                    (F.min("duration_ms")*60000).alias("Min Playlist Duration")).show()

+---------------------+---------------------+---------------------+
|Avg Playlist Duration|Max Playlist Duration|Min Playlist Duration|
+---------------------+---------------------+---------------------+
|   9.3478061056188E11|       38104427520000|           5852280000|
+---------------------+---------------------+---------------------+



In [55]:
# Q4: What is the average playlist track number? Is there a maximum amount of tracks to a playlist? min?

df_playlists.select((F.avg("num_tracks")).alias("Avg Number of Tracks"), \
                    (F.max("num_tracks")).alias("Max Number of Tracks"), \
                    (F.min("num_tracks")).alias("Min Number of Tracks")).show()

+--------------------+--------------------+--------------------+
|Avg Number of Tracks|Max Number of Tracks|Min Number of Tracks|
+--------------------+--------------------+--------------------+
|           66.346428|                 376|                   5|
+--------------------+--------------------+--------------------+

